In [2]:
import csv
def partionnage(chemin):
    row=[]
    with open(chemin, 'r') as file:
        i=0
        reader = csv.reader(file, delimiter=';')
        new_row={}
        for row in reader:
            if(i!=0):
                temp_dic={}
                temp_dic["date"]=(row[0].split(","))[1].split(" ")[0]
                temp_dic["heure"]=int((row[0].split(","))[1].split(" ")[1].split(":")[0])
                temp_dic["OT"]=float((row[0].split(","))[2])
                new_row[int((row[0].split(","))[0])]=temp_dic
            i=i+1
        return new_row
    

data=partionnage('ETTh1_without_missing.csv')
dataFinale=partionnage('ETTh1_without_missing.csv')
data={cle: valeur for i, (cle, valeur) in enumerate(data.items()) if i < len(data) - 100}


In [3]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from datetime import datetime

#sans normalisation
def predictionNeuronne(data):
    dates = [datetime.strptime(value['date'], '%Y-%m-%d') for key, value in data.items()]
    years = [date.year for date in dates]
    months = [date.month for date in dates]
    days = [date.day for date in dates]
    hours = [value['heure'] for key, value in data.items()]
    ots = [value['OT'] for key, value in data.items()]
    
    features = np.column_stack((years, months, days, hours))  # Pas de normalisation
    
    X_train, X_test, y_train, y_test = train_test_split(features, ots, test_size=0.2, random_state=42)
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
    
    loss, mae = model.evaluate(X_test, y_test)
    print("mae=",model.evaluate(X_test, y_test)[1])
    return model

def pretraitementNewValue(date,hour):
    years = int(date.split("-")[0])
    months = int(date.split("-")[1])
    days = int(date.split("-")[2])
    return  np.column_stack((years, months, days, hour))


def prediction(date,hour,data):
    
    features =pretraitementNewValue(date,hour)
    predicted_scale=predictionNeuronne(data).predict(features)
    
    return predicted_scale

#sans normalisation
print(prediction("2018-06-30",4,data))






Epoch 1/10


345/345 [==============================] - 1s 2ms/step - loss: 222.1316 - mae: 9.2916 - val_loss: 81.5384 - val_mae: 7.4717
Epoch 2/10
345/345 [==============================] - 1s 1ms/step - loss: 81.5436 - mae: 7.0815 - val_loss: 75.3019 - val_mae: 6.4280
Epoch 3/10
345/345 [==============================] - 0s 1ms/step - loss: 83.4672 - mae: 7.1578 - val_loss: 76.9635 - val_mae: 7.2534
Epoch 4/10
345/345 [==============================] - 1s 2ms/step - loss: 91.5530 - mae: 7.4910 - val_loss: 72.9210 - val_mae: 6.3542
Epoch 5/10
345/345 [==============================] - 1s 2ms/step - loss: 85.9942 - mae: 7.2547 - val_loss: 69.0610 - val_mae: 6.4953
Epoch 6/10
345/345 [==============================] - 1s 1ms/step - loss: 82.1285 - mae: 7.0603 - val_loss: 81.7628 - val_mae: 7.5472
Epoch 7/10
345/345 [==============================] - 1s 2ms/step - loss: 77.6293 - mae: 6.8756 - val_loss: 68.9260 - val_mae: 6.4381
Epoch 8/10
345/345 [==============================] - 1s

In [5]:
#en normalisant tout
def predictionNeuronne(data):
    dates = [datetime.strptime(value['date'], '%Y-%m-%d') for key, value in data.items()]
    years = [date.year for date in dates]
    months = [date.month for date in dates]
    days = [date.day for date in dates]
    hours = [value['heure'] for key, value in data.items()]
    ots = [value['OT'] for key, value in data.items()]
    
    scaler_year = StandardScaler()
    scaler_month = StandardScaler()
    scaler_day = StandardScaler()
    scaler_hour = StandardScaler()
    scaler_ot = StandardScaler()
    
    years_scaled = scaler_year.fit_transform(np.array(years).reshape(-1, 1))
    months_scaled = scaler_month.fit_transform(np.array(months).reshape(-1, 1))
    days_scaled = scaler_day.fit_transform(np.array(days).reshape(-1, 1))
    hours_scaled = scaler_hour.fit_transform(np.array(hours).reshape(-1, 1))
    ots_scaled = scaler_ot.fit_transform(np.array(ots).reshape(-1, 1))
    
    features = np.column_stack((years_scaled, months_scaled, days_scaled, hours_scaled))
    
    X_train, X_test, y_train, y_test = train_test_split(features, ots_scaled, test_size=0.2, random_state=42)
    
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
    
    loss, mae = model.evaluate(X_test, y_test)
    print("mae=",model.evaluate(X_test, y_test)[1])
    return model, scaler_year, scaler_month, scaler_day, scaler_hour, scaler_ot

def pretraitementNewValue(date, hour, scaler_year, scaler_month, scaler_day, scaler_hour):
    years = int(date.split("-")[0])
    months = int(date.split("-")[1])
    days = int(date.split("-")[2])
    hour_scaled = scaler_hour.transform([[hour]])
    return np.column_stack((scaler_year.transform([[years]]),
                            scaler_month.transform([[months]]),
                            scaler_day.transform([[days]]),
                            hour_scaled))

def prediction(date, hour, data):
    model, scaler_year, scaler_month, scaler_day, scaler_hour, scaler_ot = predictionNeuronne(data)
    features = pretraitementNewValue(date, hour, scaler_year, scaler_month, scaler_day, scaler_hour)
    predicted_scale = model.predict(features)
    # Retrouver les valeurs originales de OT
    predicted_ot = scaler_ot.inverse_transform(predicted_scale)
    return predicted_ot

print(prediction("2018-06-13", 4, data))

Epoch 1/10
345/345 [==============================] - 1s 2ms/step - loss: 0.3085 - mae: 0.4179 - val_loss: 0.1484 - val_mae: 0.2951
Epoch 2/10
345/345 [==============================] - 0s 1ms/step - loss: 0.1411 - mae: 0.2902 - val_loss: 0.1336 - val_mae: 0.2815
Epoch 3/10
345/345 [==============================] - 0s 1ms/step - loss: 0.1320 - mae: 0.2808 - val_loss: 0.1316 - val_mae: 0.2783
Epoch 4/10
345/345 [==============================] - 0s 1ms/step - loss: 0.1274 - mae: 0.2757 - val_loss: 0.1261 - val_mae: 0.2723
Epoch 5/10
345/345 [==============================] - 1s 1ms/step - loss: 0.1248 - mae: 0.2727 - val_loss: 0.1235 - val_mae: 0.2699
Epoch 6/10
345/345 [==============================] - 0s 1ms/step - loss: 0.1209 - mae: 0.2682 - val_loss: 0.1220 - val_mae: 0.2683
Epoch 7/10
345/345 [==============================] - 0s 1ms/step - loss: 0.1196 - mae: 0.2667 - val_loss: 0.1191 - val_mae: 0.2647
Epoch 8/10
345/345 [==============================] - 0s 1ms/step - loss: 0.